In [ ]:
"""
-*- coding: utf-8 -*-
========================
AWS Lambda
========================
Contributor: Aditya Tomar
========================
"""

import os
import json
import boto3
import pandas as pd
import re


def lambda_handler(event, context):

    BUCKET_NAME = os.environ["BUCKET_NAME"]
    PREFIX = os.environ["PREFIX"]

    job_id = json.loads(event["Records"][0]["Sns"]["Message"])["JobId"]
    print(event["Records"][0]["Sns"]["Message"])
    document_name = json.loads(event["Records"][0]["Sns"]["Message"])["DocumentLocation"]["S3ObjectName"][6:-4]
    print(document_name)
    # page_lines = process_response(job_id)
    df = process_response(job_id)

    csv_key_name = f"{document_name}.csv"
    # df = pd.DataFrame(page_lines.items())
    # df.columns = ["PageNo", "Text"]
    df.to_csv(f"/tmp/{csv_key_name}", index=False)

    upload_to_s3(f"/tmp/{csv_key_name}", BUCKET_NAME, f"{PREFIX}/{csv_key_name}")
    print(df)

    return {"statusCode": 200, "body": json.dumps("File uploaded successfully!")}


def upload_to_s3(filename, bucket, key):
    s3 = boto3.client("s3")
    s3.upload_file(Filename=filename, Bucket=bucket, Key=key)


def process_response(job_id):
    textract = boto3.client("textract")

    response = {}
    pages = []

    response = textract.get_document_text_detection(JobId=job_id)

    pages.append(response)

    nextToken = None
    if "NextToken" in response:
        nextToken = response["NextToken"]

    while nextToken:
        response = textract.get_document_text_detection(
            JobId=job_id, NextToken=nextToken
        )
        pages.append(response)
        nextToken = None
        if "NextToken" in response:
            nextToken = response["NextToken"]

    page_lines = {}
    for page in pages:
        for item in page["Blocks"]:
            if item["BlockType"] == "LINE":
                if item["Page"] in page_lines.keys():
                    page_lines[item["Page"]] = page_lines[item["Page"]]+ " " +item["Text"]
                else:
                    page_lines[item["Page"]] = ''
                    page_lines[item["Page"]] = page_lines[item["Page"]]+ " " +item["Text"]
    
    # comparing terms
    page_1_og = [' TERMS AND CONDITIONS','/We the Applicant/Borrower(s) hereby undertake, declare, accept and agree to abide by the following terms and conditions:', '1.', 'By signing these terms and conditions, the Borrower(s), understands, confirms , declares and agrees', 'a.', 'that the Borrower(s) has/have read and understood all the terms and conditions of the Bank as contained herein and in the', 'facility documents as regards the grant of loan/overdraft/credit facility and agree to be bound by the same;', 'b.', "that the grant of the said facility is subject to Bank's discretion and the interest will be charged on the utilized amount at a rate", 'specified by the Bank;', 'C.', 'that the end use of the funds is for approved, non-speculative purposes;', 'd.', 'that complete, true, accurate and correct information would be provided from time to time.', 'e.', 'to bear and pay the Bank, all related charges, expenses taxes, levies, etc. in respect of availing of credit facility/pledge agreement,', 'the transactions hereunder and all writings executed pursuant to or in connection with the availing of credit facility/Pledge', 'Agreement, including without limitation the expenditure incurred on the creation of any Security, enforcement of security and', 'recovery of dues and Borrower hereby authorizes the Bank to deduct/adjust these charges, expenses etc. against the security', 'held by the Bank;', 'f.', 'to comply with all the rules and regulations, terms and conditions, as prescribed from time to time by various statutory', 'authorities in connection with the security charged to the Bank and/or the credit/financial facility availed from the Bank;', 'g.', 'that the Borrower shall comply with all the requisite formalities including signing of all deeds, documents and things as may be', 'required by the Bank from time to time and as may be necessary and incidental to the availing of the credit facility.', '2.', 'The expression "Bank" shall unless it be repugnant to the context or meaning thereof, be deemed to mean and include, its successors', 'and assigns and the expression "Borrower" shall unless it be repugnant to the context or meaning thereof, be deemed to mean and', 'include, the legal heirs, executors, administrators, successors and assigns);', '3.', 'The above mentioned Gold Ornaments stand charged to the Bank as and by way of first and exclusive charge to secure, the due', 'repayment and discharge of the credit facility sanctioned/to be sanctioned, together with interest, costs, charges, expenses, fees,', 'commission (at the rates decided by the Bank from time to time) and/or any other monies due and payable, for due performance of', 'the terms and conditions, subject to which the credit facility is granted/may be granted and repayment on demand to the Bank of', 'all and every sum/s of money, which shall for the time being be owning to the Bank on any account from me/us or from any one of', 'them for the loan or any other credit/financial facility (or for any other money for which me/us or any one of them may be liable on', 'any account to the Bank, whatsoever including any/all legal charges, expenses, commissions, bank charges etc. The security created', 'herein shall not be determined by any change in the constitution, winding up, insanity, bankruptcy, liquidation, insolvency, death,', 'disability or incapacity whatsoever and howsoever of I/We the Borrower(s) cum Pledgor(s).', '4.', 'I/We declare and assure that all the above mentioned Gold Ornaments pledged/proposed to be pledged to the Bank are the', 'absolute properties of the Borrower/Pledgor, at the sole disposal of the Borrower/Pledgor and free from any, claims.', '5.', 'The Borrower hereby authorizes the bank to carry out the NEFT/DD/Fund Transfer transaction as per detail mentioned in the', 'application. I/we understand that RTGS/NEFT/DD/Fund Transfer request is subject to the RBI Regulation and guidelines governing', 'the same. I/we further acknowledge that the bank accepts no liability for any consequences arising out of erroneous details provided', 'by me/us.', '6.', 'The Borrower hereby specifically agree/agrees that the Bank shall be entitled to change/vary the interest rates at its discretion and', 'on account of any change as may be directed by Reserve Bank of India and/or any other regulatory /statutory body from time to', 'time. The Borrower agree/agrees to pay interest at such revised rates as aforesaid. It is agreed that it will not be necessary for the', 'Bank to send intimation of change of rate of interest to the Borrower and a publication either in the news paper or on the notice', 'board will be sufficient and the Borrower will be deemed to have consented to such change. Interest shall be charged on the', 'outstanding(s) in the accounts opened in respect of the Facilities at such rate(s) as may be determined by the Bank from time to time', "at the Bank's sole discretion provided that the rate(s) shall be subject to changes in the Bank's Benchmark Base Rate and/or changes", 'in interest rates prescribed by the Reserve Bank of India from time to time.', '7.', 'If the Borrower fails to carry out and perform any of the obligations under this Application/Pledge agreement/facility documents or', 'commits breach of any of the terms hereof or of the Sanction Letter /Pledge Agreement/Facility Documents or if any of the', 'representations, warranties, assurances, statements and particulars contained in the Request letter/this application form being', "found in the Bank's opinion (which shall be conclusive and binding on the Borrower) to be incorrect, or if after execution hereof, any", 'circumstance shall occur which in the sole judgment of the Bank is prejudicial to or imperils or is likely to prejudice or imperil the', 'Credit Facility and the securities created thereunder, then the Bank shall, if it thinks fit, be entitled at the risk and expense of the', 'Borrower after giving notice, at any time or time after such occurrence and in its sole judgment to refuse to grant the said Facility or', 'any part thereof which may remain to be disbursed or to recall the said Facility and/or to enforce the security that may be/is required', 'to be created thereunder. On the question whether any of the events, matters or circumstances mentioned above have or has', 'occurred, the opinion of the Bank shall be final, conclusive and binding on the Borrower, both in and out of court/ judicial/ quasi', 'judicial authority. It shall not be open to the Borrower at any stage to question the decision of the Bank to refuse to grant the said', 'Facilities or any part thereof which may remain to be disbursed or to recall the said Facility and/or to enforce the security created', 'thereunder.', '8.', 'The Bank reserves the right to vary, alter, modify or rescind, amend or change at any time any one or more of the terms and', "conditions of the facility at the Bank' discretion with such notice as the Bank may deem reasonable and without assigning any", 'reasons.', '9.', 'The Borrower is aware of and acknowledges and agrees that under the Agreement for Pledge, the Bank is entitled to and that the', "Bank may, assign, transfer, sell, securitize, novate or part with (whether in full or in part) the Bank's rights and liabilities under this", "Agreement (including the right to receive all or any portion of the Pledgor's Dues) to one or more persons and that the Bank may", "also assign and transfer all or some of its rights and obligations hereunder (including the security hereunder for the Borrower's Dues", 'and this right to assign hereunder) to one or more such persons. In such case the Borrower shall become the primary borrower of', 'such assignee/buyer and shall at no point of time raise any objection, legal or otherwise regarding assignment, transfer and sale of', 'the securities by you in favour of the assignee / buyer.', '10.', 'The Borrower agrees that in addition to any other rights available to the Bank, in the event of the Borrower committing any default,', 'the Bank shall be entitled to disclose to the Reserve Bank of India or any other statutory/regulatory authority or to any other third', 'person, the name/identity of the Borrower and/or all its/their Directors and the default as committed. In case of default in payment', '3']
    page_2_og = [' of any of the Borrower’s Dues to the Bank, the Bank or the Reserve Bank of India will have an unqualified right to disclose or publish', 'the name of the /Borrower and/or its/ directors/partners as defaulters (including as willful defaulters) in such manner and through', 'such medium as the Bank and/or the Reserve Bank of India in their absolute discretion may deem fit.', '11.', 'The Borrower hereby consents to the disclosure by the Bank, of information and data relating to the Borrower/partners/directors,', 'of the Facilities availed or to be availed by the Borrower, obligations assumed or to be assumed by the Pledgor/Borrower in relation', 'thereto and default, if any, committed by the Borrower in discharge thereof, as the Bank may deem appropriate and necessary, to', 'Credit Information Bureau (India).', '12.', 'The Valuation of the gold security shall be done by an independent assayer appointed by the bank and the loan amount/credit', 'facility shall be determined by the Bank on the basis of the value of the Gold security as set out in the valuation report supplied by', 'the assayer. Such value shall be subject to the market price of the Gold security from time to time. The amount of the loan will be', 'conveyed to me/us by and under a sanction letter. I/we agree to abide by the terms and conditions of the sanction letter.', '13.', 'Upon maturity, I/we shall either repay the loan/credit facility along with interest, and such other dues as are payable by me/us to the', 'Bank ("Dues") or I/we may, at any time request a renewal/ enhancement of the loan which the Bank may grant us at its sole and', 'absolute discretion, subject to the continuation of pledge over the Gold security in favour of the bank and provision of such other', 'security and in such from as the bank may require and such variation in the rate of interest as the bank may prescribe as per its', 'policies prevailing at the time of such renewal/ enhancement.', '14.', 'Any of the following shall constitute an Event of Default under this Agreement', 'a.', 'Delay in interest service of more than the specified number of days or such other number of days as may be specified from time', 'to time by the Bank.', 'b.', 'Overdrawing in the account.', 'C.', 'Non-churning/no credit entries in account for over 2 month.', 'd.', 'Non-review of account beyond three months of due date.', 'e.', 'Non-repayment of limits after the due date.', 'f.', 'any representation made by the Borrower to the Bank which shall be found by the Bank to have been false at any time or', 'misleading as of the date on which the same was made or deemed to be made;', 'g.', 'Breach of any terms and conditions, obligations, representations contained herein and/or in the sanction letter, security', 'documents, facility documents etc.', 'h.', 'Occurrence of any circumstances or event which adversely affects my/our capacity to repay the loan or to honour the obligations', 'or which is prejudicial to the interest of the bank.', 'i.', 'Non-repayment of EMI after the due date.', 'j.', 'Failure to comply with margin call beyond the specified number of days or such other number of days as may be specified', 'by the Bank from time to time, from the date of call.', 'k.', 'Deterioration or impairment of any security or any part thereof which may impact the value/security in the opinion of the', 'bank.', 'I.', 'Non-repayment of loan after the due date.', 'm.', 'Commission/detection of any fraud as regards the gold ornaments pledged with the Bank or otherwise.', 'n.', 'Suspicion of any fraud/error or Erroneous/fraudulent valuation of the security pledged by reason of any collusion between', 'me/us and the assayer.', 'o.', 'Arrest or detention in prison on the ground of theft of ornaments or any suspicion of theft of ornaments.', 'p.', 'in the event of death of any Borrower, insolvency, bankruptcy.', 'q.', 'Such other event as may be specified by the Bank in the Sanction Letter or otherwise.', 'In the event of any default by the Borrower in duly repaying/ paying the dues or any part thereof and/ or failure in the Borrower/', 'Pledgor performing its obligations in accordance with the provisions of the Pledge Agreement/Transaction/Facility document the', 'Bank shall, without the requirement of any consent or concurrence of the Borrower/Pledgor, be entitled to do all or any of the', 'following:', 'a)', 'Upon the occurrence of any of event, the dues shall immediately become payable and the Bank shall be fully entitled to', 'recall the loan/credit facility.', 'b)', 'invoke and enforce the pledge;', 'c)', 'sell or otherwise dispose of all or any of the Pledged Commodities by public auction or private contract or in such manner, at', 'such price and on such terms and subject to such conditions as the Bank may think fit.', 'd)', 'Sell the Pledged Gold Ornaments to the Assayer.', 'e)', 'The Bank shall not in any way be liable for any loss or depreciation in value of the Pledged Gold Ornaments whilst in the', 'possession of the Bank or in the course of sale, realisation or disposal or otherwise.', 'f)', 'Such invocation and sale of the Pledged Gold Ornaments shall be to the cost of the Borrower/Pledgor and any shortfall arising', 'out of such invocation and sale shall be made good to the Bank by the Borrower/Pledgor at his/its sole costs, risks and', 'responsibility.', '15.', 'That in the event of the death of Borrower, the Bank may return the gold ornaments to the Nominee appointed by the Borrower,', 'subject to the Bank receiving its entire dues from the Nominee. In the event the Nominee fails to make the payment when intimated', 'by the Bank, the Bank may return the ornaments to any of the legal heir who has paid all the dues of the Bank. If the Legal heirs fail', 'to make the payment when demanded, the Bank may dispose of the goods in the manner deemed fit and the excess money if any', 'may be paid to the Nominee and in the absence of nomination to the legal heirs. The shortfall of any amount may be recovered from', "any of the Borrower's properties.", '16.', 'Unless the same falls within the jurisdiction of the Debts Recovery Tribunal established under the Recovery of Debts Due To Banks', 'and Financial Institutions Act, 1993, any and all claims and disputes arising out of or in connection with this Application, Pledge', 'Agreement etc. or its performance shall be settled by arbitration by a single Arbitrator to be appointed by the Bank. The arbitration', 'shall be held in Mumbai.', '4']
    full_terms = ''

    for line in page_1_og:
        full_terms = full_terms + " " + line
    
    for line in page_2_og:
        full_terms = full_terms + " " + line
    full_terms = full_terms.replace("â€™", "'")

    full_terms_cp = ''
    full_terms_cp = page_lines[3]+' '+page_lines[4]
    
    print(page_2_og)
    
    
    
    full_terms = re.split('; |: |\. |\*|\n', full_terms)
    full_terms_cp = re.split('; |: |\. |\*|\n', full_terms_cp)
    
    print(full_terms)
    print(full_terms_cp)
    
    diff_lines = {}
    i = 0
    for sentence in list(set(full_terms_cp) - set(full_terms)):
        if sentence not in [" TERMS AND CONDITIONS /We the Applicant/Borrower(s) hereby undertake, declare, accept and agree to abide by the following terms and conditions", "In case of default in payment 3  of any of the Borrower's Dues to the Bank, the Bank or the Reserve Bank of India will have an unqualified right to disclose or publish the name of the /Borrower and/or its/ directors/partners as defaulters (including as willful defaulters) in such manner and through such medium as the Bank and/or the Reserve Bank of India in their absolute discretion may deem fit"]:
            diff_lines[i] = sentence
            i = i+1
            
    
    words = []
    for sentence in diff_lines.values():
        words.append(sentence.split(' ')[0])
    print(words)
    # print(diff_lines)
    
    
    # print(list(set(full_terms) - set(full_terms_cp)))
    
    
    
    coordinates_data = []
    for page in pages:
        for item in page["Blocks"]:
            if item["BlockType"] == "WORD":
                row=[]
                row.append(item['Text'])
                row.append(item['Geometry']['Polygon'][0]['X'])
                row.append(item['Geometry']['Polygon'][0]['Y'])
                row.append(item['Geometry']['Polygon'][2]['X'])
                row.append(item['Geometry']['Polygon'][2]['Y'])
                row.append(item['Page'])
                coordinates_data.append(row)

    coordinates_data = pd.DataFrame(coordinates_data, columns=['word', 'x1', 'y1', 'x2', 'y2', 'page'])
    coordinates_data = coordinates_data[(coordinates_data['page']<5) & (coordinates_data['page']>2) ]
    
    # for i in range(coordinates_data.shape[0]):
    #     if str(coordinates_data.iloc[i, 0]).replace('.', '').replace(':', '').replace(';', '') in words:
            
    print(coordinates_data)
            

    
    # for page in pages:
    #     for item in page["Blocks"]:
    #         if item["BlockType"] == "WORD" and item['text'] in words:
    #             for sentence in  
    
    # page_lines = {}
    # for page in pages:
    #     for item in page["Blocks"]:
    #         if item["BlockType"] == "LINE":
    #             if item["Page"] in page_lines.keys():
    #                 page_lines[item["Page"]] = page_lines[item["Page"]]+ " " +item["Text"]
    #             else:
    #                 page_lines[item["Page"]] = ''
    #                 page_lines[item["Page"]] = page_lines[item["Page"]]+ " " +item["Text"]
    return coordinates_data[coordinates_data.apply(lambda row: True if (row['word'] in words) else False, axis = 1)]